In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch/minsearch.py

--2025-06-11 05:20:12--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5820 (5.7K) [text/plain]
Saving to: ‘minsearch.py.4’

minsearch.py.4      100%[===================>]   5.68K  --.-KB/s    in 0s      

2025-06-11 05:20:12 (64.4 MB/s) - ‘minsearch.py.4’ saved [5820/5820]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []
for course_dict in docs_raw :
    for doc in course_dict['documents'] :
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
index.fit(documents)

In [9]:
from openai import OpenAI

In [10]:
client = OpenAI()

In [33]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEST when answring the QUESTION
if the CONTEXT doesn't contain the answer, output NONE
QUESTION: the course has already started, can I still enroll

CONTEXT :
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final

In [63]:
def search(query) :
    boost = {'question':3.0, 'section': 0.5}

    results =index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results
    

In [64]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEST when answring the QUESTION
    
    QUESTION: {question}
    
    CONTEXT :
    {context}
    """.strip()

    context = ""

    for doc in search_results :
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [65]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user", "content" :  prompt}]
    )
    
    return response.choices[0].message.content

In [66]:
def rag(query)  :
    query = query
    results = search(query)
    prompt = build_prompt(query,results)
    answer = llm(prompt)
    return answer

In [67]:
rag("The course just started, can I still enroll the course ?")

"Yes, you can still enroll in the course even after it has started. You are eligible to submit the homeworks. However, keep in mind that there will be deadlines for submitting the final projects, so it's important not to leave everything until the last minute."